In [410]:
#all library connection
import pandas as pd
import numpy as np
import re #регулярки
from math import gcd, isclose
import sympy as sp #библиотека символьных вычеслений, мат.задачи
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [411]:
# pymatgen def to obtain structures
from pymatgen.core.structure import Composition

In [412]:
from collections import defaultdict

In [413]:
#loading data
df = pd.read_csv(r'C:\Users\Ксения\Project\Magnet_Exchange_bias\data\Bias_data.csv')

In [414]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 53 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   id                                              533 non-null    int64  
 1   crystal_structure(core/shell)                   533 non-null    object 
 2   Space group(core)                               532 non-null    object 
 3   Space group(shell)                              533 non-null    object 
 4   chemical formula                                533 non-null    object 
 5   Structure                                       533 non-null    object 
 6   Structure_                                      533 non-null    object 
 7   length (D)                                      521 non-null    float64
 8   width                                           521 non-null    float64
 9   depth                                      

***Difference***
- composition - core+shell with Ni1O1 like formula
-  chemical formula - Ni50NiO50
-  Structure - alhpa, gamma

## Поработаем со столбцами

In [415]:
# Удалим все колонки, значения которых точно не подлежат востановлению
del_col = ['Exp type', 'Scale bar (nm)', 'microphotography pic','Spectrum', 'JCPDS card (core)', 'JCPDS card (shell)', 'X-ray tube (nm)', 'Instrument', 'Crystalline size', 'Material fraction', 'curie temperature (K)', 'H-range min (kOe)', 'Instrument.1', 'M-H curve', 'applied field(kOe)', 'Article id', 'Method', 'H-range min (kOe)','Tb', 'Keff (effective anisotropy constant) (erg/cm3)',
            'Keff (Jm^-3)', 'Total magnetization ', 'Atomic mass ratio', 'Number of magnetic ions per unit cell', 'Name', 'synonyms',
            'get_exchange_group_info', 'cooling fields (mT)', 'Hc of measurement (kOe) ', 'number_of_unique_magnetic_sites', 'number_of_magnetic_sites', 'Cas rn (core_1)', 'Cas rn (core_2)' ]
df = df.drop(columns=del_col)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 533 non-null    int64  
 1   crystal_structure(core/shell)      533 non-null    object 
 2   Space group(core)                  532 non-null    object 
 3   Space group(shell)                 533 non-null    object 
 4   chemical formula                   533 non-null    object 
 5   Structure                          533 non-null    object 
 6   Structure_                         533 non-null    object 
 7   length (D)                         521 non-null    float64
 8   width                              521 non-null    float64
 9   depth                              509 non-null    float64
 10  shape                              482 non-null    object 
 11  core_1                             533 non-null    object 

In [416]:
# Теперь можно удалить эти колонки, так как они имеют такие же значения как и crystal_structure(core/shell) и chemical formula
print(f'Unique cores: {df["core_1"].unique()}', f'Amount of unique cores composition: {len(df["core_1"].unique())}', sep='\n')
print(f'Unique shells: {df["core_2(shell)"].unique()}', f'Amount of unique shells composition: {len(df["core_2(shell)"].unique())}', sep='\n')

Unique cores: ['Zn1O1' 'Cu1O1' 'Mn0.50Zn0.50Fe2O4' 'La1Fe1O3' 'Ni1' 'Mg1Fe2O4'
 'Co1Fe2O4' 'Bi1Fe1O3' 'Co1' 'Fe1Rh1' 'Mn1O2' 'Ag1' 'Fe1' 'Fe1O1' 'Ni1O1'
 'Fe3O4' 'Bi2Fe4O9' 'Ti1O2' 'Co0.4Fe2.6O4' 'Co0.7Fe2.3O4' 'Fe0.7Co0.3'
 'Fe0.5Co0.5' 'Fe0.3Co0.7' 'La2Fe1Mn1O6' 'Fe1V1O4' 'Au1'
 'Bi0.9Gd0.1Fe0.9Ti0.1O3' 'Nd0.78Sr0.22CoO3' 'Fe2O3' 'La0.85Sr0.15FeO3'
 'Mn3O4' 'La0.67Sr0.33MnO3' 'Mn1O1' 'Co3O4' 'Ni0.99Fe0.01O1'
 'Ni0.98Fe0.02O1' 'Ni0.96Fe0.04O1' 'Ni0.97Fe0.03O1' 'Ag0.5Cr2.5O4'
 'Ag1Cr1O2' 'Co1O1' 'Mn1Fe2O4' 'Bi0.9La0.1FeO3' 'Nd0.90Ho0.10FeO3'
 'Dy1Cr1Ti1O5' 'Gd1Cr1Ti1O5' 'Ba1Fe12O19' 'Ni1Fe2O4' 'Co1Cr2O4' 'Fe1Ni1'
 'Mn1' 'Fe1F2' 'Fe5C2' 'Fe1Ni3' 'Fe2.2C1' 'La2Fe2O6' 'Co1P1']
Amount of unique cores composition: 57
Unique shells: ['Co1Fe2O4' '0' 'Ni1O1' 'Bi1Fe1O3' 'Co1O1' 'Fe3O4' 'Fe2O3' 'Bi2Fe4O9'
 'Сu1O1' 'Si1O2' 'Co3O4' 'Mn3O4' 'Ni1Fe2O4' 'Co1Mn2O4' 'Fe1' 'Mn1O1'
 'Fe3C1' 'Mg1O1' 'Mg1O1, Co1Fe2O4' 'Mo1' 'Co1']
Amount of unique shells composition: 21


In [417]:
df = df.drop(columns=['core_1', 'core_2(shell)']) #удалим их

In [418]:
#переименуем название колонк для удобства и преобразуем их в нижний регистр
df.columns = df.columns.str.lower()

new_column_names = {
    'id': 'id', 'crystal_structure(core/shell)': 'crystal_structure',
    'space group(core)': 'space_group_core', 'space group(shell)': 'space_group_shell',
    'chemical formula': 'chemical_formula', 'structure': 'structure',
    'length (d)': 'length_d_nm', 'width': 'width_nm',
    'depth': 'depth_nm', 'shape': 'shape', 'temperature (k)': 'temperature_k',
    'h-range max(koe)': 'h_range_max_koe', 'ms (emu/g)': 'sat_em_g',
    'hc (oe)': 'coer_oe', 'mrem (emu/g)': 'rem_emu_g', 'exchange bias shift hb or heb(oe)': 'exc_bias_oe',
    'vertical loop shift m vsl (emu/g)': 'ver_shift_emu_g', 'fc field(t)': 'fc_field_t'
}

# Переименование столбцов
df = df.rename(columns=new_column_names)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 533 non-null    int64  
 1   crystal_structure  533 non-null    object 
 2   space_group_core   532 non-null    object 
 3   space_group_shell  533 non-null    object 
 4   chemical_formula   533 non-null    object 
 5   structure          533 non-null    object 
 6   structure_         533 non-null    object 
 7   length_d_nm        521 non-null    float64
 8   width_nm           521 non-null    float64
 9   depth_nm           509 non-null    float64
 10  shape              482 non-null    object 
 11  temperature_k      528 non-null    float64
 12  h_range_max_koe    526 non-null    float64
 13  sat_em_g           420 non-null    object 
 14  coer_oe            510 non-null    object 
 15  mr (emu/g)         430 non-null    float64
 16  exc_bias_oe        500 non

## Поработаем с химическими формулами

In [419]:
chemical_formulas = []

for formula in df['chemical_formula']: # Итерация по каждой формуле в столбце 'chemical formula'
    if not formula.isalnum(): # Проверка, содержит ли формула только буквенно-цифровые символы
        line = re.split("-|,|/|@|–", formula) # Разделение формулы по символам '-', '@', '/', and ','
        components = [] # Создание пустого списка для хранения компонентов формулы
        for i in line:  # Итерация по каждому компоненту в разделенной формуле
            if i.strip(): # Добавление компонента в список, если он не пустой
                components.append(i.strip())
        chemical_formulas.append(components) # Добавление списка компонентов в список химических формул
    else:
        chemical_formulas.append([formula])  # Если формула состоит только из буквенно-цифровых символов, добавляем ее как есть

df['chemical_formula'] = chemical_formulas # Замена столбца новыми значениями

In [420]:
# Функция для разбивки списка в каждой ячейке на отдельные элементы и вставки их в новые столбцы
def split_elements(row):
    # Получаем список элементов из ячейки
    elements = row['chemical_formula']
    # Размещаем элементы в соответствующих столбцах
    for i, element in enumerate(elements):
        if i < 3:
            row[f'c{i+1}'] = element
    return row

# Применяем функцию к каждой строке датафрейма
df = df.apply(split_elements, axis=1)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   c1                 533 non-null    object 
 1   c2                 322 non-null    object 
 2   c3                 9 non-null      object 
 3   chemical_formula   533 non-null    object 
 4   coer_oe            510 non-null    object 
 5   crystal_structure  533 non-null    object 
 6   depth_nm           509 non-null    float64
 7   exc_bias_oe        500 non-null    float64
 8   fc_field_t         508 non-null    float64
 9   h_range_max_koe    526 non-null    float64
 10  id                 533 non-null    int64  
 11  length_d_nm        521 non-null    float64
 12  mr (emu/g)         430 non-null    float64
 13  sat_em_g           420 non-null    object 
 14  shape              482 non-null    object 
 15  space_group_core   532 non-null    object 
 16  space_group_shell  533 non

In [421]:
# Проверим как разделились значения списка
print(df['c1'], df['c2'], df['c3'], end='\n') # В с3 должно было быть всего 9 значений

0                    ZnO
1                    CoO
2      Mn0.50Zn0.50Fe2O4
3      Mn0.50Zn0.50Fe2O4
4      Mn0.50Zn0.50Fe2O4
             ...        
528               LaFeO3
529               LaFeO3
530               LaFeO3
531               LaFeO3
532               LaFeO3
Name: c1, Length: 533, dtype: object 0      CoFe2O4
1      CoFe2O4
2          NaN
3          NaN
4          NaN
        ...   
528        NiO
529        NiO
530        NiO
531        NiO
532        NiO
Name: c2, Length: 533, dtype: object 0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
528    NaN
529    NaN
530    NaN
531    NaN
532    NaN
Name: c3, Length: 533, dtype: object


In [422]:
print(df['c2'].info())

<class 'pandas.core.series.Series'>
RangeIndex: 533 entries, 0 to 532
Series name: c2
Non-Null Count  Dtype 
--------------  ----- 
322 non-null    object
dtypes: object(1)
memory usage: 4.3+ KB
None


In [423]:
#Сделаем костыль для нестеохео соединений 
print(df['c1'].unique(), df['c2'].unique(), df['c3'].unique(), end='\n')
non_steh = ['Mn0.50Zn0.50Fe2O4',  'Co0.4Fe2.6O4', 'Co0.7Fe2.3O4', 'Fe0.7Co0.3', 'Fe0.5Co0.5',
 'Fe0.3Co0.7', 'Bi0.9Gd0.1Fe0.9Ti0.1O3', 'Nd0.78Sr0.22CoO3' 'Cu0.98' 'Cu0.96', 'La0.85Sr0.15FeO3', 
 'La0.67Sr0.33MnO3', 'Ni0.99Fe0.01O', 'Ni0.98Fe0.02O', 'Ni0.96Fe0.04O', 'Ni0.97Fe0.03O', 'Ag0.5Cr2.5O4',
 'Bi0.9La0.1FeO3', 'Fe2.2C', 'Fe0.02O', 'Fe0.04O']

['ZnO' 'CoO' 'Mn0.50Zn0.50Fe2O4' 'LaFeO3' 'Ni' 'MgFe2O4' 'CoFe2O4'
 'BiFeO3' 'Co' 'FeRh' 'MnO2' 'Ag' 'Fe' 'FeO' 'NiO' 'Fe3O4' 'Bi2Fe4O9'
 'TiO2' 'Co0.4Fe2.6O4' 'Co0.7Fe2.3O4' 'Fe0.7Co0.3' 'Fe0.5Co0.5'
 'Fe0.3Co0.7' 'La2FeMnO6' 'FeVO4' 'Au' 'Bi0.9Gd0.1Fe0.9Ti0.1O3'
 'Nd0.78Sr0.22CoO3' 'Cu0.98' 'Cu0.96' 'Fe2O3' 'La0.85Sr0.15FeO3' 'Mn3O4'
 'La0.67Sr0.33MnO3' 'MnO' 'Co3O4' 'Ni0.99Fe0.01O' 'Ni0.98Fe0.02O'
 'Ni0.96Fe0.04O' 'Ni0.97Fe0.03O' 'Ag0.5Cr2.5O4' 'AgCrO2' 'MnFe2O4'
 'Bi0.9La0.1FeO3' 'NdFeO3' 'DyCrTiO5' 'GdCrTiO5' 'BaFe12O19' 'NiFe2O4'
 'CoCr2O4' 'FeNi' 'Mn' 'Fe5C2' 'FeNi3' 'Fe2.2C' 'La2Fe2O6' 'FeF2' 'CoP'] ['CoFe2O4' nan 'NiO' 'BiFeO3' 'CoO' 'Fe3O4' 'Fe2O3' 'Bi2Fe4O9' 'Fe0.02O'
 'Fe0.04O' 'Co3O4' 'Mn3O4' 'NiFe2O4' 'CoMn2O4' 'MnO' 'Fe3C' 'MgO' 'Mo'
 'Co' 'Fe'] [nan 'CoFe2O4']


In [424]:
# Костыль для таких соединений
non_stoich_compounds_dict = {
    'Mn0.50Zn0.50Fe2O4': 'MnZnFe4O8', 'Co0.4Fe2.6O4': 'Co2Fe13O20',
    'Co0.7Fe2.3O4': 'Co7Fe23O40', 'Fe0.7Co0.3': 'Fe7Co3', 'Fe0.5Co0.5': 'FeCo',
    'Fe0.3Co0.7': 'Fe3Co7', 'Bi0.9Gd0.1Fe0.9Ti0.1O3': 'Bi9GdFe9TiO30',
    'Nd0.78Sr0.22CoO3': 'Nd39Sr11Co50O150', 'Cu0.98': 'Cu98', 'Cu0.96': 'Cu96',
    'La0.85Sr0.15FeO3': 'La17Sr3Fe20O60', 'La0.67Sr0.33MnO3': 'La67Sr33Mn100O300',
    'Ni0.99Fe0.01O': 'Ni99FeO100', 'Ni0.98Fe0.02O': 'Ni49FeO50', 'Ni0.96Fe0.04O': 'Ni24FeO25',
    'Ni0.97Fe0.03O': 'Ni97Fe3O100', 'Ag0.5Cr2.5O4': 'AgCr5O8', 'Bi0.9La0.1FeO3': 'Bi9LaFe10O30',
    'Fe2.2C': 'Fe11C5', 'Fe0.02O': 'FeO50', 'Fe0.04O': 'FeO25'
}

In [425]:
def replace_non_stoichiometric(column, non_stoich_compounds_dict):
    '''Функция заменяет значения ключей на значения для этих ключей'''
    for i, value in enumerate(column):
        if value != 0 and value in non_stoich_compounds_dict:
            column.at[i] = non_stoich_compounds_dict[value]
    return column

In [426]:
# Применение функции к каждому столбцу
new_df_c1 = replace_non_stoichiometric(df['c1'], non_stoich_compounds_dict)
new_df_c2 = replace_non_stoichiometric(df['c2'], non_stoich_compounds_dict)
new_df_c3 = replace_non_stoichiometric(df['c3'], non_stoich_compounds_dict)

# Вывод уникальных значений после применения функции
unique_values_c1 = new_df_c1.unique()
unique_values_c2 = new_df_c2.unique()
unique_values_c3 = new_df_c3.unique()

In [427]:
print(f"Уникальные значения в столбце c1 после применения функции:{unique_values_c1}\nУникальные значения в столбце c2 после применения функции:{unique_values_c2}\nУникальные значения в столбце c3 после применения функции:{unique_values_c3}", end='\n')

Уникальные значения в столбце c1 после применения функции:['ZnO' 'CoO' 'MnZnFe4O8' 'LaFeO3' 'Ni' 'MgFe2O4' 'CoFe2O4' 'BiFeO3' 'Co'
 'FeRh' 'MnO2' 'Ag' 'Fe' 'FeO' 'NiO' 'Fe3O4' 'Bi2Fe4O9' 'TiO2'
 'Co2Fe13O20' 'Co7Fe23O40' 'Fe7Co3' 'FeCo' 'Fe3Co7' 'La2FeMnO6' 'FeVO4'
 'Au' 'Bi9GdFe9TiO30' 'Nd39Sr11Co50O150' 'Cu98' 'Cu96' 'Fe2O3'
 'La17Sr3Fe20O60' 'Mn3O4' 'La67Sr33Mn100O300' 'MnO' 'Co3O4' 'Ni99FeO100'
 'Ni49FeO50' 'Ni24FeO25' 'Ni97Fe3O100' 'AgCr5O8' 'AgCrO2' 'MnFe2O4'
 'Bi9LaFe10O30' 'NdFeO3' 'DyCrTiO5' 'GdCrTiO5' 'BaFe12O19' 'NiFe2O4'
 'CoCr2O4' 'FeNi' 'Mn' 'Fe5C2' 'FeNi3' 'Fe11C5' 'La2Fe2O6' 'FeF2' 'CoP']
Уникальные значения в столбце c2 после применения функции:['CoFe2O4' nan 'NiO' 'BiFeO3' 'CoO' 'Fe3O4' 'Fe2O3' 'Bi2Fe4O9' 'FeO50'
 'FeO25' 'Co3O4' 'Mn3O4' 'NiFe2O4' 'CoMn2O4' 'MnO' 'Fe3C' 'MgO' 'Mo' 'Co'
 'Fe']
Уникальные значения в столбце c3 после применения функции:[nan 'CoFe2O4']


### Время для использования Composition

Однако в наших колонках есть пропуски и 0, поэттому надо создать функцию, которая будет игнорировать такие строки

In [428]:
def process_column(df, column_name):
    """
    Функция проверяет каждую ячейку в столбце DataFrame на равенство 0 или NaN.
    Если значение не равно 0 или NaN, оно заменяется на результат применения класса Composition из библиотеки pymatgen.

    Args:
        df (pandas.DataFrame): Исходный DataFrame.
        column_name (str): Имя столбца, который нужно обработать.

    Returns:
        pandas.DataFrame: DataFrame c обработанным столбцом.
    """
    new_column = []
    for value in df[column_name]:
        if pd.notna(value) and value != 0:
            # Если значение не NaN и не равно 0, применяем класс Composition
            comp = Composition(str(value))
            new_value = comp.formula
        else:
            new_value = value
        new_column.append(new_value)

    df[column_name] = new_column
    return df

Теперь у меня есть работающая функция composition для столбцов с nan и 0

In [429]:
# Применим на наши колонки c1, c2, c3
processed_df = process_column(df, column_name='c1')
processed_df = process_column(df, column_name='c2')
processed_df = process_column(df, column_name='c3')
print(df['c3'].unique())
print(df['c2'].unique())

[nan 'Fe2 Co1 O4']
['Fe2 Co1 O4' nan 'Ni1 O1' 'Fe1 Bi1 O3' 'Co1 O1' 'Fe3 O4' 'Fe2 O3'
 'Fe4 Bi2 O9' 'Fe1 O50' 'Fe1 O25' 'Co3 O4' 'Mn3 O4' 'Fe2 Ni1 O4'
 'Mn2 Co1 O4' 'Mn1 O1' 'Fe3 C1' 'Mg1 O1' 'Mo1' 'Co1' 'Fe1']


Создадим отдельный столбец, где будем содержать полностью материал созданный из трёх колонок c1, c2, c3

In [430]:
# Функция для сложения формул из нескольких столбцов
def combine_compositions(row):
    compositions = []
    for column in ['c1', 'c2', 'c3']:
        value = row[column]
        if pd.notnull(value):  # Проверяем, не пустое ли значение
            compositions.append(value)
    return ' '.join(compositions)

# Создаем новый столбец с объединенными формулами
df['formula_combined'] = df.apply(combine_compositions, axis=1)

In [431]:
print(df['formula_combined']) # Получилось немного криво, но не критично я думаю

0      Zn1 O1 Fe2 Co1 O4
1      Co1 O1 Fe2 Co1 O4
2         Mn1 Zn1 Fe4 O8
3         Mn1 Zn1 Fe4 O8
4         Mn1 Zn1 Fe4 O8
             ...        
528    La1 Fe1 O3 Ni1 O1
529    La1 Fe1 O3 Ni1 O1
530    La1 Fe1 O3 Ni1 O1
531    La1 Fe1 O3 Ni1 O1
532    La1 Fe1 O3 Ni1 O1
Name: formula_combined, Length: 533, dtype: object


## Теперь поработаем с space_group_core и space_group_shell

In [432]:
# Преобразование значений в нижний регистр на всякий случай
df['space_group_core'] = df['space_group_core'].str.lower()
df['space_group_shell'] = df['space_group_shell'].str.lower()

In [433]:
#посмотрим какие структуры у нас есть
print(f"Уникальные значение ядер: {df['space_group_core'].unique()}")
print(f"Уникальные значения оболочек: {df['space_group_shell'].unique()}")

Уникальные значение ядер: ['p63mc' 'fm-3m' 'fd-3m' 'pnma' 'r3c' 'p63/mmc' 'pm-3m' 'i4/m' 'im-3m'
 'pbam' 'p42/mnm' 'p1' nan 'i41/amd' 'p-3m1' 'r3m' 'p4332' 'p4/mmm'
 'i-43m' 'cmcm' 'p4132' 'c2/c' 'p6322' 'pnnm']
Уникальные значения оболочек: ['fd-3m' '0' 'fm-3m' 'r3c' 'p4332' 'pbam' 'p4132' 'pnma' 'i41/amd'
 'p41212' 'im-3m' 'p63/mmc' 'p42/mnm']


[ 'fm-3m' 'fd-3m' 'pbnm' - заменить на pnma 'r3c''pm-3m' 'im-3m' 'i-43m'
  'p-3m1'

In [434]:
# Надо заменить на значения с чертой/ - если значение равно 221-230 или 215-220 или 200-206 или 187-190 или 162-167

Осторожней с 62 - так она может пиcаться ещё как 'Pnma' и 'pbnm'

In [435]:
#создадим словарь, чтобы перевести простаранственные группы в числовые обозначения пространнственных групп
dict_space_group = {
    1: "P1", 2: "P-1", 3: "P121", 4: "P1211", 5: "C121", 6: "P1m1", 7: "P1c1", 8: "C1m1",
    9: "C1c1", 10: "P12/m1", 11: "P121/m1", 12: "C12/m1", 13: "P12/c1", 14: "P121/c1",
    15: "C2/c", 16: "P222", 17: "P2221", 18: "P21212", 19: "P212121", 20: "C2221",
    21: "C222", 22: "F222", 23: "I222", 24: "I212121", 25: "Pmm2", 26: "Pmc21", 27: "Pcc2",
    28: "Pma2", 29: "Pca21", 30: "Pnc2", 31: "Pmn21", 32: "Pba2", 33: "Pna21", 34: "Pnn2",
    35: "Cmm2", 36: "Cmc21", 37: "Ccc2", 38: "Amm2", 39: "Abm2", 40: "Ama2", 41: "Aba2",
    42: "Fmm2", 43: "Fdd2", 44: "Imm2", 45: "Iba2", 46: "Ima2", 47: "Pmmm", 48: "Pnnn",
    49: "Pccm", 50: "Pban", 51: "Pmma", 52: "Pnna", 53: "Pmna", 54: "Pcca", 55: "Pbam",
    56: "Pccn", 57: "Pbcm", 58: "Pnnm", 59: "Pmmn", 60: "Pbcn", 61: "Pbca", 62: "Pnma",
    63: "Cmcm", 64: "Cmca", 65: "Cmmm", 66: "Cccm", 67: "Cmma", 68: "Ccca", 69: "Fmmm",
    70: "Fddd", 71: "Immm", 72: "Ibam", 73: "Ibca", 74: "Imma", 75: "P4", 76: "P41",
    77: "P42", 78: "P43", 79: "I4", 80: "I41", 81: "P-4", 82: "I-4", 83: "P4/m", 84: "P42/m",
    85: "P4/n", 86: "P42/n", 87: "I4/m", 88: "I41/a", 89: "P422", 90: "P4212", 91: "P4122",
    92: "P41212", 93: "P4222", 94: "P42212", 95: "P4322", 96: "P43212", 97: "I422",
    98: "I4122", 99: "P4mm", 100: "P4bm", 101: "P42cm", 102: "P42nm", 103: "P4cc",
    104: "P4nc", 105: "P42mc", 106: "P42bc", 107: "I4mm", 108: "I4cm", 109: "I41md",
    110: "I41cd", 111: "P-42m", 112: "P-42c", 113: "P-421m", 114: "P-421c", 115: "P-4m2",
    116: "P-4c2", 117: "P-4b2", 118: "P-4n2", 119: "I-4m2", 120: "I-4c2", 121: "I-42m",
    122: "I-42d", 123: "P4/mmm", 124: "P4/mcc", 125: "P4/nbm", 126: "P4/nnc", 127: "P4/mbm",
    128: "P4/mnc", 129: "P4/nmm", 130: "P4/ncc", 131: "P42/mmc", 132: "P42/mcm",
    133: "P42/nbc", 134: "P42/nnm", 135: "P42/mbc", 136: "P42/mnm", 137: "P42/nmc",
    138: "P42/ncm", 139: "I4/mmm", 140: "I4/mcm", 141: "I41/amd", 142: "I41/acd",
    143: "P3", 144: "P31", 145: "P32", 146: "R3", 147: "P-3", 148: "R-3", 149: "P312",
    150: "P321", 151: "P3112", 152: "P3121", 153: "P3212", 154: "P3221", 155: "R32",
    156: "P3m1", 157: "P31m", 158: "P3c1", 159: "P31c", 160: "R3m", 161: "R3c", 162: "P-31m",
    163: "P-31c", 164: "P-3m1", 165: "P-3c1", 166: "R-3m", 167: "R-3c", 168: "P6", 169: "P61",
    170: "P65", 171: "P62", 172: "P64", 173: "P63", 174: "P-6", 175: "P6/m", 176: "P63/m",
    177: "P622", 178: "P6122", 179: "P6522", 180: "P6222", 181: "P6422", 182: "P6322",
    183: "P6mm", 184: "P6cc", 185: "P63cm", 186: "P63mc", 187: "P-6m2", 188: "P-6c2",
    189: "P-62m", 190: "P-62c", 191: "P6/mmm", 192: "P6/mcc", 193: "P63/mcm", 194: "P63/mmc",
    195: "P23", 196: "F23", 197: "I23", 198: "P213", 199: "I213", 200: "Pm-3", 201: "Pn-3",
    202: "Fm-3", 203: "Fd-3", 204: "Im-3", 205: "Pa-3", 206: "Ia-3", 207: "P432", 208: "P4232",
    209: "F432", 210: "F4132", 211: "I432", 212: "P4332", 213: "P4132", 214: "I4132",
    215: "P-43m", 216: "F-43m", 217: "I-43m", 218: "P-43n", 219: "F-43c", 220: "I-43d",
    221: "Pm-3m", 222: "Pn-3n", 223: "Pm-3n", 224: "Pn-3m", 225: "Fm-3m", 226: "Fm-3c",
    227: "Fd-3m", 228: "Fd-3c", 229: "Im-3m", 230: "Ia-3d"
}


In [436]:
# Создание словаря со значениями в нижнем регистре для сравнени по нему
dict_space_group_lower = {key: value.lower() for key, value in dict_space_group.items()}

In [437]:
# Чтобы проще было проверять и менять значения
dict_space_group_swapped = {v: k for k, v in dict_space_group_lower.items()}

In [438]:
print(dict_space_group_swapped)

{'p1': 1, 'p-1': 2, 'p121': 3, 'p1211': 4, 'c121': 5, 'p1m1': 6, 'p1c1': 7, 'c1m1': 8, 'c1c1': 9, 'p12/m1': 10, 'p121/m1': 11, 'c12/m1': 12, 'p12/c1': 13, 'p121/c1': 14, 'c2/c': 15, 'p222': 16, 'p2221': 17, 'p21212': 18, 'p212121': 19, 'c2221': 20, 'c222': 21, 'f222': 22, 'i222': 23, 'i212121': 24, 'pmm2': 25, 'pmc21': 26, 'pcc2': 27, 'pma2': 28, 'pca21': 29, 'pnc2': 30, 'pmn21': 31, 'pba2': 32, 'pna21': 33, 'pnn2': 34, 'cmm2': 35, 'cmc21': 36, 'ccc2': 37, 'amm2': 38, 'abm2': 39, 'ama2': 40, 'aba2': 41, 'fmm2': 42, 'fdd2': 43, 'imm2': 44, 'iba2': 45, 'ima2': 46, 'pmmm': 47, 'pnnn': 48, 'pccm': 49, 'pban': 50, 'pmma': 51, 'pnna': 52, 'pmna': 53, 'pcca': 54, 'pbam': 55, 'pccn': 56, 'pbcm': 57, 'pnnm': 58, 'pmmn': 59, 'pbcn': 60, 'pbca': 61, 'pnma': 62, 'cmcm': 63, 'cmca': 64, 'cmmm': 65, 'cccm': 66, 'cmma': 67, 'ccca': 68, 'fmmm': 69, 'fddd': 70, 'immm': 71, 'ibam': 72, 'ibca': 73, 'imma': 74, 'p4': 75, 'p41': 76, 'p42': 77, 'p43': 78, 'i4': 79, 'i41': 80, 'p-4': 81, 'i-4': 82, 'p4/m': 8

In [439]:
def replace_space_group_values(df, dict_space_group_swapped):
    for column in ['space_group_core', 'space_group_shell']:
        for i, value in enumerate(df[column]):
            if pd.notnull(value) and value != 0:
                # Проверяем наличие значения в словаре и заменяем его, если оно есть
                if value in dict_space_group_swapped:
                    df.at[i, column] = dict_space_group_swapped[value]
    return df

# Применяем функцию к датафрейму
new_df = replace_space_group_values(df, dict_space_group_swapped)

In [440]:
# Теперь все наши значения закодированы - не все есть моменты, которые стоит исправить - может там русские буквы
print(f"Для ядра: {df['space_group_core'].unique()}", f"Для оболочки: {df['space_group_shell'].unique()}", sep='\n')

Для ядра: [186 225 227 62 161 194 221 87 229 55 136 1 nan 141 164 160 212 123 217 63
 213 15 182 58]
Для оболочки: [227 '0' 225 161 212 55 213 62 141 92 229 194 136]


In [441]:
# Для проверки первого столбца
dict_crystal_structure = {
    "triclinic":[1, 2], 
    "monoclinic":[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    "orthorhombic":[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74], 
    "tetragonal":[75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142], 
    "trigonal":[143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167], 
    "hexagonal":[168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194],
    "cubic":[195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230]
}               

# Геометрические признаки 

Посмотрим на парметры формы и на x/y/z(nm) параметры частиц.
1. Закодируем названия форм, но сначала удалим одни и те же формы, но написанные по разному
2. Найдём площадь и объем наших частиц
3. Посмотрим на сферичность
4. Посмотрим на разброс классов

In [442]:
print(df.info()) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   c1                 533 non-null    object 
 1   c2                 322 non-null    object 
 2   c3                 9 non-null      object 
 3   chemical_formula   533 non-null    object 
 4   coer_oe            510 non-null    object 
 5   crystal_structure  533 non-null    object 
 6   depth_nm           509 non-null    float64
 7   exc_bias_oe        500 non-null    float64
 8   fc_field_t         508 non-null    float64
 9   h_range_max_koe    526 non-null    float64
 10  id                 533 non-null    int64  
 11  length_d_nm        521 non-null    float64
 12  mr (emu/g)         430 non-null    float64
 13  sat_em_g           420 non-null    object 
 14  shape              482 non-null    object 
 15  space_group_core   532 non-null    object 
 16  space_group_shell  533 non

In [443]:
print(df['shape'].unique()) # Как можно заметить многие формы являются одним и тем же, сгруппируем

['sphere' 'polyhedron' 'tetrahedral' 'octahedral' nan 'wire' 'flower'
 'egg' 'rod' 'quasi-spherical' 'cube' 'octopod' 'octahedral ' 'cubic'
 'triangle' 'hexahedron' 'quasi-sphere' 'polyhedral' 'pseudo-spherical'
 'pseudospherically' 'hexagonal' 'octahedron']


In [444]:
# Составим словарь для форм
dict_shape = { 1: ['sphere', 'quasi-spherical', 'quasi-sphere', 'pseudo-spherical', 'pseudospherically'], 
               2: ['octahedral', 'octahedron', 'octahedral '], 
               3: ['polyhedron', 'polyhedral'], 
               4: ['cubic', 'cube']
               5: ['hexahedron', 'hexagonal']
               6: ['tetrahedral', 'triangle']
               7: ['wire', 'rod']
}

НУЖНО ПОНЯТЬ К ЧЕМУ ОТНОСИТь egg и flower

In [ ]:
# Перевеём значения формы в цифры

# Поработаем с пропусками

In [445]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   c1                 533 non-null    object 
 1   c2                 322 non-null    object 
 2   c3                 9 non-null      object 
 3   chemical_formula   533 non-null    object 
 4   coer_oe            510 non-null    object 
 5   crystal_structure  533 non-null    object 
 6   depth_nm           509 non-null    float64
 7   exc_bias_oe        500 non-null    float64
 8   fc_field_t         508 non-null    float64
 9   h_range_max_koe    526 non-null    float64
 10  id                 533 non-null    int64  
 11  length_d_nm        521 non-null    float64
 12  mr (emu/g)         430 non-null    float64
 13  sat_em_g           420 non-null    object 
 14  shape              482 non-null    object 
 15  space_group_core   532 non-null    object 
 16  space_group_shell  533 non

## Поработаем с геометрическими признаками